# ✈️ FlightOnTime — Predição de Atrasos em Voos

O **FlightOnTime** é um projeto de *Data Science* e *Back-End* voltado para prever a probabilidade de um voo **decolar no horário ou com atraso**, utilizando dados históricos de aviação civil.  
Ele faz parte de um desafio educacional cujo objetivo é construir um **MVP preditivo** capaz de apoiar passageiros, companhias aéreas e aeroportos na tomada de decisão.

## Descrição do projeto

### 🎯 Objetivo do Projeto

Criar um modelo de classificação binária que, a partir de informações de um voo — como companhia aérea, origem, destino, horário e distância — estima se ele será **Pontual (0)** ou **Atrasado (1)**, retornando também a probabilidade associada.

---

### 🧠 Pipeline de Data Science

O notebook conduz as principais etapas do fluxo de ciência de dados:

1. **ETL e limpeza dos dados**  
   - Carregamento, padronização e enriquecimento das informações de voos.

2. **Engenharia de variáveis (Feature Engineering)**  
   - Extração de atributos relevantes, como:  
     hora do voo, dia da semana, distância, aeroportos, companhia aérea.

3. **Modelagem preditiva**  
   - Testes com algoritmos supervisionados (ex.: Logistic Regression, Random Forest).

4. **Avaliação do modelo**  
   - Métricas como Acurácia, Precisão, Recall e F1-score.

5. **Exportação do modelo treinado**  
   - Serialização via `joblib` para uso pela API de Back-End.

---

### 🛠️ Integração com a API

O modelo gerado neste notebook será consumido por uma **API REST em Java (Spring Boot)**, que disponibilizará o endpoint:

```
POST /predict
```

Retornando:

```json
{
  "previsao": "Atrasado",
  "probabilidade": 0.78
}
```

---

### 📌 Sobre o Dataset

O projeto utiliza um conjunto de dados contendo informações reais de voos, incluindo:

- companhia aérea  
- aeroporto de origem e destino  
- data e horário de partida  
- distância do trecho  
- status de atraso  

As funções de ETL foram previamente desenvolvidas para garantir limpeza, consistência e rastreabilidade.

## 🚀 Início do Notebook

A partir daqui, o notebook executa o fluxo completo de preparação, análise e modelagem, resultando em um modelo leve, funcional e pronto para produção no MVP do FlightOnTime.

### Imports Globais

In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
from utils.utils import parse_categoricals, parse_datetime

import automlx
from automlx import init # type: ignore

### Preprocessamento e Carga

Execute as funções abaixo para carregar os dados de fonte local ou para baixar, preprocessar e salvar os dados a partir da base de dados de vôos da ANAC (Agência Nacional de Aviação Civil).  
- Os dados são salvos em ./dados/  
- A função `processar_dados` acessa toda a base de dados da ANAC a partir de Janeiro/2001 até Outubro/2025.

In [3]:
def processar_dados(save: bool = True) -> pd.DataFrame:
    """
    Executa o pipeline completo de ETL dos dados de voos a partir da base de dados de vôos da ANAC (Agência Nacional de Aviação Civil).

    Baixa os arquivos CSV brutos, realiza o pré-processamento, consolida os dados
    em um único DataFrame e salva automaticamente os resultados em formatos
    CSV e Parquet no diretório ./data/, com versionamento por timestamp.

    Parâmetros
    ----------
    **save** : bool, opcional
        Indica se o DataFrame resultante deve ser salvo em disco.
        Padrão é True, o que significa que os arquivos serão salvos
        automaticamente.

    Retorna
    -------
    pd.DataFrame
        Dataset consolidado e pré-processado.
    """
    from etl.etl import getUrls, preprocess_csvs
        
    urls = getUrls()
    dataset = preprocess_csvs(urls)
    if save:
      from etl.etl import save_df
      save_df(dataset, timestamp=True)

    return dataset   


def carregar_dados(filename: str) -> pd.DataFrame:
    """
    Carrega um dataset de voos previamente pré-processado a partir de um arquivo Parquet
    e aplica conversões de tipo para colunas categóricas e de data/hora.

    Parâmetros
    ----------
    filename : str
        Nome-base do arquivo Parquet localizado em ./data/.
        Não inclua a extensão ".parquet".

    Retorna
    -------
    pd.DataFrame
        Dataset carregado com colunas categóricas convertidas para `category`
        e colunas de data/hora convertidas para `datetime`, pronto para análise
        ou modelagem.
    """
    # Função interna para converter colunas categóricas
    # def parse_categoricals(df: pd.DataFrame) -> pd.DataFrame:
    #     categorical_columns = [
    #         "Empresa Aérea",
    #         "Código Tipo Linha",
    #         "Aeródromo Origem",
    #         "Aeródromo Destino",
    #     ]
    #     for col in categorical_columns:
    #         df[col] = df[col].astype('category')
        
    #     return df
    
    # # Função interna para converter colunas de data/hora
    # def parse_datetime(df: pd.DataFrame) -> pd.DataFrame:
    #     datetime_columns = [
    #         "Data Hora Voo",
    #     ]
    #     for col in datetime_columns:
    #         df[col] = pd.to_datetime(df[col], format="mixed", dayfirst=True, errors='coerce')
        
    #     return df
    
    filepath = f'{os.getcwd()}/data/{filename}.parquet'
    print(f"Carregando dataset local de: ./data/{filename}.parquet")
    dataset = pd.read_parquet(filepath)
    dataset = parse_categoricals(dataset)
    dataset = parse_datetime(dataset)
    print("🏁 Dataset carregado com sucesso!")

    return dataset

#### Preprocessar com origem externa

Baixa e executa o preprocessamento em tempo de execução através da base de dados abertos da ANAC.  
Link: https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VRA%29/  

Para mais informações sobre o processo, consulte a documentação do módulo etl.

In [ ]:
# Código comentado por segurança
# Descomente a linha abaixo para executar o pipeline ETL completo

# df = processar_dados()

Iniciando o download e preprocessamento de 310 arquivos CSV...

[1/310] Carregando: http://.../2000/01%20-%20Janeiro/VRA_20001.csv
✔ 0 linhas carregadas.
   Total atual de linhas: 0
   Memória usada: 0.00 MB

[2/310] Carregando: http://.../2000/02%20-%20Fevereiro/VRA_20002.csv
✔ 0 linhas carregadas.
   Total atual de linhas: 0
   Memória usada: 0.00 MB

[3/310] Carregando: http://.../2000/03%20-%20Mar%C3%A7o/VRA_20003.csv
✔ 0 linhas carregadas.
   Total atual de linhas: 0
   Memória usada: 0.00 MB

[4/310] Carregando: http://.../2000/04%20-%20Abril/VRA_20004.csv
✔ 0 linhas carregadas.
   Total atual de linhas: 0
   Memória usada: 0.00 MB

[5/310] Carregando: http://.../2000/05%20-%20Maio/VRA_20005.csv
✔ 6599 linhas carregadas.
   Total atual de linhas: 6599
   Memória usada: 0.16 MB

[6/310] Carregando: http://.../2000/06%20-%20Junho/VRA_20006.csv
✔ 7792 linhas carregadas.
   Total atual de linhas: 14391
   Memória usada: 0.34 MB

[7/310] Carregando: http://.../2000/07%20-%20Julho/VRA_

#### Preprocessar com origem local

Exemplo de nome de arquivo local:  
```python
filename = "dados_voos_20250101_153000"
```

- Não necessita informar o caminho do arquivo e extensão.  
- Caminho default: ./data
- Extensão default: .parquet

In [5]:
filename = "dados_voos_20251227_140820"
df = carregar_dados(filename=filename)

Carregando dataset local de: ./data/dados_voos_20251227_140820.parquet
🏁 Dataset carregado com sucesso!


In [ ]:
df.shape

(10371507, 6)

#### Visualizações

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10371507 entries, 0 to 10371506
Data columns (total 6 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Empresa Aérea      category      
 1   Código Tipo Linha  category      
 2   Aeródromo Origem   category      
 3   Aeródromo Destino  category      
 4   Data Hora Voo      datetime64[ns]
 5   Atrasado           int8          
dtypes: category(4), datetime64[ns](1), int8(1)
memory usage: 148.4 MB


In [8]:
df

,Empresa Aérea,Código Tipo Linha,Aeródromo Origem,Aeródromo Destino,Data Hora Voo,Atrasado
0,PEP,R,SBHT,SBBE,2000-05-01 07:40:00.000,1
1,PEP,R,SBSN,SBHT,2000-05-01 11:45:00.000,1
2,PEP,R,SWJN,SWJU,2000-05-01 08:45:00.000,1
3,BLC,N,SBBR,SBCT,2000-05-01 18:18:00.000,1
4,NES,R,SBSP,SBIP,2000-05-01 16:10:00.000,1
...,...,...,...,...,...,...
10371502,AZU,N,SBSV,SBKP,2025-10-02 02:50:00.100,0
10371503,TAM,N,SBFZ,SBGL,2025-10-02 18:20:00.000,1
10371504,TAM,N,SBBR,SBRJ,2025-10-24 09:35:00.000,0
10371505,GLO,N,SBBE,SBBR,2025-10-22 05:00:00.000,0


#### Refinar ETL

In [9]:
def check_nulls(df: pd.DataFrame, col: str) -> None:
    nulls_notice = f'Coluna "{col}" possui {df[col].isna().sum()} valores nulos.'
    print(nulls_notice)
    return

def check_uniques(df: pd.DataFrame, col: str) -> None:
    print(f'Coluna "{col}":')
    print(f'{df[col].unique()}\n')
    return

##### Verificando Nulos

In [10]:
for col in df.columns:
    check_nulls(df, col)

Coluna "Empresa Aérea" possui 0 valores nulos.
Coluna "Código Tipo Linha" possui 0 valores nulos.
Coluna "Aeródromo Origem" possui 0 valores nulos.
Coluna "Aeródromo Destino" possui 0 valores nulos.
Coluna "Data Hora Voo" possui 0 valores nulos.
Coluna "Atrasado" possui 0 valores nulos.


##### Verificando Uniques

In [11]:
for col in df.columns:
    check_uniques(df, col)

Coluna "Empresa Aérea":
['PEP', 'BLC', 'NES', 'PTN', 'TAM', ..., 'SID', 'IPM', 'OMI', 'ASO', 'ACN']
Length: 46
Categories (46, object): ['ABJ', 'ACN', 'AMG', 'AMS', ..., 'VRG', 'VRN', 'VSP', 'WEB']

Coluna "Código Tipo Linha":
['R', 'N', 'H']
Categories (3, object): ['H', 'N', 'R']

Coluna "Aeródromo Origem":
['SBHT', 'SBSN', 'SWJN', 'SBBR', 'SBSP', ..., 'SN6L', 'SNSS', 'MDPC', 'SBCO', 'SBUY']
Length: 316
Categories (316, object): ['2NHT', 'LPPT', 'MDPC', 'SAAR', ..., 'SWTS', 'SWTU', 'SWVC', 'SWYN']

Coluna "Aeródromo Destino":
['SBBE', 'SBHT', 'SWJU', 'SBCT', 'SBIP', ..., 'SNLN', 'SNSS', 'SMJP', 'SBCO', 'SBUY']
Length: 318
Categories (318, object): ['2NHT', 'MDPC', 'SAAR', 'SACO', ..., 'SWTU', 'SWVC', 'SWYN', 'SYCJ']

Coluna "Data Hora Voo":
<DatetimeArray>
[       '2000-05-01 07:40:00',        '2000-05-01 11:45:00',
        '2000-05-01 08:45:00',        '2000-05-01 18:18:00',
        '2000-05-01 16:10:00',        '2000-05-01 23:28:00',
        '2000-05-01 20:33:00',        '2000-05-0

##### Eliminando os Nulos

In [ ]:
df = df.dropna(subset="Código Tipo Linha")
check_nulls(df, "Código Tipo Linha")

#### Engenharia de Features

Nesta seção transformamos as colunas `Partida Prevista` e `Partida Real` em features separadas:
- `Data do Voo` *datetime64[ns]*  
    - Data em que o vôo ocorreu.  

- `Horario_min` *int16*  
    - Horário do dia em que o vôo ocorreu em *minutos*.  
    *0 = 00:00, 1439 = 23:59*
- `Dia da Semana` *int8*  
    - Dia da semana em que o vôo ocorreu.  
    *0 = Segunda, 6 = Domingo*

A variável dependente (X) `Atrasado` é gerada a partir da comparação entre `Partida Prevista` e `Partida Real`.  

Por fim, eliminamos as variáveis originais `Partida Prevista` e `Partida Real`.

##### Data do Vôo

In [ ]:
# Ativa o modo copy-on-write do pandas para evitar SettingWithCopyWarning
# pd.options.mode.copy_on_write = True

# df['Data do Voo'] = df['Data Hora Voo'].dt.date
# df['Data do Voo'] = pd.to_datetime(df['Data do Voo'], dayfirst=True, errors='coerce') # Transforma em Datetime

##### Horário do Vôo

In [ ]:
# 0 = 0:00, 1439 = 23:59
# df['Horario_min'] = (
#     df['Data Hora Voo'].dt.hour * 60 + # type: ignore
#     df['Data Hora Voo'].dt.minute # type: ignore
# ).astype('int16')

##### Dia da Semana

In [ ]:
# 0 = Segunda, 6 = Domingo
# df['Dia da Semana'] = df['Data Hora Voo'].dt.weekday.astype('int8') # type: ignore

##### Verificando

In [ ]:
df.info()

In [ ]:
df

In [ ]:
for col in df.columns:
    check_nulls(df, col)

#### Visualizando (Y)

Aqui analisamos a distribuição dos vôos atrasados no histórico de vôos observados.  

- Aproximadamente 8.5 milhões de vôos foram pontuais.  
- Aproximadamente 5 milhões de vôos atrasaram, totalizando aproximadamente 36.45% dos vôos observados.  

In [ ]:
from utils.plot_feature import plot_feature
print(df['Atrasado'].value_counts())
print(df['Atrasado'].value_counts(normalize=True))
plot_feature(df, 'Atrasado')

#### Separando X e Y

Aqui separaremos as features (X) da variável alvo (Y).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['Atrasado']),
    df['Atrasado'],
    test_size=0.3,
    random_state=42,
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.head()

In [ ]:
y_train.head()

#### Modelagem

Para o processo de modelagem, decidimos usar a moderna biblioteca [Oracle AutoMLx](https://docs.oracle.com/en-us/iaas/tools/automlx/latest/latest/index.html), que automatiza os processos de `Seleção de Algoritmo`, `Sampling Adaptativo`, `Seleção de Features` e `Ajuste de Hiperparâmetros`, resultando num processo facilitado e com melhores métricas no modelo final.  

In [ ]:
init(engine='local')

##### Referência de Pipeline:
Deletar depois **

```python
custom_pipeline = automlx.Pipeline(
    task='classification',
    model_list=[                 # Specify the models you want the AutoMLx to consider
        'LogisticRegression',
        'LGBMClassifier',
        'GaussianNB'
    ],
    n_algos_tuned=2,             # Choose how many models to tune
    min_features=[               # Specify minimum features to force the model to use. It can take 3 possible types of values:
        'native-country',        # If int, 0 < min_features <= n_features,
        'marital-status',              # If float, 0 < min_features <= 1.0, 1.0 means disabling feature selection
        'education-num'          # If list, names of features to keep, for example ['a', 'b'] means keep features 'a' and 'b'
    ],
    adaptive_sampling=False,     # Disable or enable Adaptive Sampling step. Default to `True`
    preprocessing=True,          # Disable or enable Preprocessing step. Default to `True`
    search_space={               # You can specify the hyper-parameters and ranges we search
        'LGBMClassifier': {
            'learning_rate': {'range': [0.01, 10], 'type': 'continuous'},
            'boosting_type': {'range': ['gbdt', 'dart'], 'type': 'categorical'},
        },
    },
    max_tuning_trials=2,         # The maximum number of tuning trials. Can be integer or Dict (max number for each model)
    score_metric='f1_macro',     # Any scikit-learn metric or a custom function
)

est1 = custom_pipeline.fit(
    X_train,
    y_train,
    # X_val,
    # y_val,
    time_budget= 20,    # Specify time budget in seconds
    cv='auto'           # Automatically pick a good cross-validation (cv) strategy for the user's dataset.
                        # Ignored if X_valid and y_valid are provided.
                        # Can also be:
                        #   - An integer (For example, to use 5-fold cross validation)
                        #   - A list of data indices to use as splits (for advanced, such as time-based splitting)
)
```

##### Custom Pipeline

In [ ]:
# Referência de Pipeline:
# Deletar depois **

# custom_pipeline = automlx.Pipeline(
#     task='classification',
#     model_list=[                 # Specify the models you want the AutoMLx to consider
#         'LogisticRegression',
#         'LGBMClassifier',
#         'GaussianNB'
#     ],
#     n_algos_tuned=2,             # Choose how many models to tune
#     min_features=[               # Specify minimum features to force the model to use. It can take 3 possible types of values:
#         'native-country',        # If int, 0 < min_features <= n_features,
#         'marital-status',              # If float, 0 < min_features <= 1.0, 1.0 means disabling feature selection
#         'education-num'          # If list, names of features to keep, for example ['a', 'b'] means keep features 'a' and 'b'
#     ],
#     adaptive_sampling=False,     # Disable or enable Adaptive Sampling step. Default to `True`
#     preprocessing=True,          # Disable or enable Preprocessing step. Default to `True`
#     search_space={               # You can specify the hyper-parameters and ranges we search
#         'LGBMClassifier': {
#             'learning_rate': {'range': [0.01, 10], 'type': 'continuous'},
#             'boosting_type': {'range': ['gbdt', 'dart'], 'type': 'categorical'},
#         },
#     },
#     max_tuning_trials=2,         # The maximum number of tuning trials. Can be integer or Dict (max number for each model)
#     score_metric='f1_macro',     # Any scikit-learn metric or a custom function
# )

# est1 = custom_pipeline.fit(
#     X_train,
#     y_train,
#     # X_val,
#     # y_val,
#     time_budget= 20,    # Specify time budget in seconds
#     cv='auto'           # Automatically pick a good cross-validation (cv) strategy for the user's dataset.
#                         # Ignored if X_valid and y_valid are provided.
#                         # Can also be:
#                         #   - An integer (For example, to use 5-fold cross validation)
#                         #   - A list of data indices to use as splits (for advanced, such as time-based splitting)
# )

##### Default Pipeline

In [ ]:
# est1 = automlx.Pipeline(task='classification', score_metric='recall') # type: ignore
# est1.fit(X_train, y_train)

In [ ]:
y_proba = est1.predict_proba(X_test)
score_default = roc_auc_score(y_test, y_proba[:, 1])

print(f'Roc_auc score em dados de teste: {score_default}')

#### Análise do Processo de otimização do AutoMLx

Nesta seção analisamos o processo de otimização executado pelo AutoMLx de forma a compreender melhor o processo de modelagem e garantir a qualidade do resultado obtido.  

Para mais informações acesse:
- [Documentação do AutoMLx](https://docs.oracle.com/en-us/iaas/tools/automlx/latest/latest/index.html)  
- [Demo: AutoMLx Classification](https://github.com/oracle-samples/automlx/blob/main/demos/OracleAutoMLx_Classification.ipynb)

##### Sumário

In [ ]:
est1.print_summary()

##### Seleção de Algoritmo

In [ ]:
# Each trial is a row in a dataframe that contains
# Algorithm, Number of Samples, Number of Features, Hyperparameters, Score, Runtime, Memory Usage, Step as features
trials = est1.completed_trials_summary_[est1.completed_trials_summary_["Step"].str.contains('Model Selection')]
name_of_score_column = f"Score ({est1._inferred_score_metric[0].name})"
trials.replace([np.inf, -np.inf], np.nan, inplace=True)
trials.dropna(subset=[name_of_score_column], inplace = True)
scores = trials[name_of_score_column].tolist()
models = trials['Algorithm'].tolist()
colors = []

y_margin = 0.10 * (max(scores) - min(scores))
s = pd.Series(scores, index=models).sort_values(ascending=False)
s = s.dropna()
for f in s.keys():
    if f.strip()  ==  est1.selected_model_.strip():
        colors.append('orange')
    elif s[f] >= s.mean():
        colors.append('teal')
    else:
        colors.append('turquoise')

fig, ax = plt.subplots(1)
ax.set_title("Algorithm Selection Trials")
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.set_ylabel(est1._inferred_score_metric[0].name)
s.plot.bar(ax=ax, color=colors, edgecolor='black')
ax.axhline(y=s.mean(), color='black', linewidth=0.5)
plt.show()
plt.close()

##### Sampling Adaptativo

In [ ]:
# Each trial is a row in a dataframe that contains
# Algorithm, Number of Samples, Number of Features, Hyperparameters, Score, Runtime, Memory Usage, Step as features
trials = est1.completed_trials_summary_[est1.completed_trials_summary_["Step"].str.contains('Adaptive Sampling')]
trials.replace([np.inf, -np.inf], np.nan, inplace=True)
trials.dropna(subset=[name_of_score_column], inplace = True)
scores = trials[name_of_score_column].tolist()
n_samples = [int(sum(d.values()) / len(d)) if isinstance(d, dict) else d for d in trials['# Samples']]


y_margin = 0.10 * (max(scores) - min(scores))
fig, ax = plt.subplots(1)
ax.set_title("Adaptive Sampling ({})".format(est1.selected_model_))
ax.set_xlabel('Dataset sample size')
ax.set_ylabel(est1._inferred_score_metric[0].name)
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(n_samples, scores, 'k:', marker="s", color='teal', markersize=3)
plt.show()
plt.close()

##### Seleção de Features

In [ ]:
print(f"Features selected: {est1.selected_features_names_}")
dropped_features = df.drop(est1.selected_features_names_raw_, axis=1).columns
print(f"Features dropped: {dropped_features.to_list()}")

# Each trial is a row in a dataframe that contains
# Algorithm, Number of Samples, Number of Features, Hyperparameters, Score, Runtime, Memory Usage, Step as features
trials = est1.completed_trials_summary_[est1.completed_trials_summary_["Step"].str.contains('Feature Selection')]
trials.replace([np.inf, -np.inf], np.nan, inplace=True)
trials.dropna(subset=[name_of_score_column], inplace = True)
trials.sort_values(by=['# Features'],ascending=True, inplace = True)
scores = trials[name_of_score_column].tolist()
n_features = trials['# Features'].tolist()

y_margin = 0.10 * (max(scores) - min(scores))
fig, ax = plt.subplots(1)
ax.set_title("Feature Selection Trials")
ax.set_xlabel("Number of Features")
ax.set_ylabel(est1._inferred_score_metric[0].name)
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(n_features, scores, 'k:', marker="s", color='teal', markersize=3)
ax.axvline(x=len(est1.selected_features_names_), color='orange', linewidth=2.0)
plt.show()
plt.close()

##### Ajuste de Hiperparâmetros

In [ ]:
# Each trial is a row in a dataframe that contains
# Algorithm, Number of Samples, Number of Features, Hyperparameters, Score, Runtime, Memory Usage, Step as features
trials = est1.completed_trials_summary_[est1.completed_trials_summary_["Step"].str.contains('Model Tuning')]
trials.replace([np.inf, -np.inf], np.nan, inplace=True)
trials.dropna(subset=[name_of_score_column], inplace = True)
trials.drop(trials[trials['Finished'] == -1].index, inplace = True)
trials['Finished']= trials['Finished'].apply(lambda x: time.mktime(datetime.datetime.strptime(x,
                                             "%a %b %d %H:%M:%S %Y").timetuple()))
trials.sort_values(by=['Finished'],ascending=True, inplace = True)
scores = trials[name_of_score_column].tolist()
score = []
score.append(scores[0])
for i in range(1,len(scores)):
    if scores[i]>= score[i-1]:
        score.append(scores[i])
    else:
        score.append(score[i-1])
y_margin = 0.10 * (max(score) - min(score))

fig, ax = plt.subplots(1)
ax.set_title("Hyperparameter Tuning Trials")
ax.set_xlabel("Iteration $n$")
ax.set_ylabel(est1._inferred_score_metric[0].name)
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(score) - y_margin, max(score) + y_margin)
ax.plot(range(1, len(trials) + 1), score, 'k:', marker="s", color='teal', markersize=3)
plt.show()
plt.close()

##### Matriz de Confusão

In [ ]:
y_pred = est1.predict(X_test)
cm = confusion_matrix(y_test.astype(int), y_pred, labels=[False, True])
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["Não Atrasou", "Atrasou"]
)

print(classification_report(y_test, y_pred))
disp.plot(cmap="viridis")

In [ ]:
(0.77 + 0.58) / 2